In [1]:
#from functions import extract_LAI_from_RAS_file, explore_image, extract_all_LAI_from_RAS_file
import matplotlib.pyplot as plt
#import torch
import numpy as np
#datapath = './dataset/france/lai_ras/'
import random
import glob
import tifffile
filepaths = glob.glob('/home/luser/UniBw-STELAR/dataset/france2/processed_lai_npy/*.npy')
filepaths.sort()

In [2]:
import torch
device = torch.device("cuda")# if torch.cuda.is_available() else "cpu")

vista_crop_dict = { 0:'NA' , 1: 'ALFALFA', 2: 'BEET', 3: 'CLOVER', 4: 'FLAX', 5: 'FLOWERING_LEGUMES', 6: 'FLOWERS', 7: 'FOREST', 8: 'GRAIN_MAIZE', 9: 'GRASSLAND', 10: 'HOPS', 11: 'LEGUMES', 12: 'VISTA_NA', 13: 'PERMANENT_PLANTATIONS', 14: 'PLASTIC', 15: 'POTATO', 16: 'PUMPKIN', 17: 'RICE', 18: 'SILAGE_MAIZE', 19: 'SOY', 20: 'SPRING_BARLEY', 21: 'SPRING_OAT', 22: 'SPRING_OTHER_CEREALS', 23: 'SPRING_RAPESEED', 24: 'SPRING_RYE', 25: 'SPRING_SORGHUM', 26: 'SPRING_SPELT', 27: 'SPRING_TRITICALE', 28: 'SPRING_WHEAT', 29: 'SUGARBEET', 30: 'SUNFLOWER', 31: 'SWEET_POTATOES', 32: 'TEMPORARY_GRASSLAND', 33: 'WINTER_BARLEY', 34: 'WINTER_OAT', 35: 'WINTER_OTHER_CEREALS', 36: 'WINTER_RAPESEED', 37: 'WINTER_RYE', 38: 'WINTER_SORGHUM', 39: 'WINTER_SPELT', 40: 'WINTER_TRITICALE', 41: 'WINTER_WHEAT'}

In [4]:
chosen_crop_types = 1
print("vista_crop_dict[chosen_crop_types]", vista_crop_dict[chosen_crop_types])


vista_crop_dict[chosen_crop_types] ALFALFA


In [37]:
per_crop_spatio_temporal_data_size = 850
labels = np.load('/home/luser/stelar_3d/storage/full_mast/vista_labes_aligned.npy').astype(np.uint8)
x_inds, y_inds = np.where(labels==chosen_crop_types)

print("len(x_inds)", len(x_inds))
print("len(y_inds)", len(y_inds))

len(x_inds) 1106626
len(y_inds) 1106626


In [41]:
x_inds, y_inds

(array([   6,    6,    7, ..., 9995, 9995, 9995]),
 array([8042, 8043, 8040, ..., 7110, 7111, 7112]))

In [43]:
random_corner = len(x_inds)-1
random_corner

1106625

In [48]:
x_corner = x_inds[random_corner]
x_corner

9995

In [52]:
if(x_corner > labels.shape[0]-70):
    x_corner = x_corner-70

print(x_corner)
print(x_corner+64)

9925
9989


In [31]:
temporal_batches = torch.tensor([]).to(device)
spatial_label_batches = torch.tensor([]).to(device)
for i in range(per_crop_spatio_temporal_data_size):
    time_strip_ind =  np.random.randint(0, len(filepaths)-65)
    #time_strip_ind = 0
    #time_strip_ind = len(filepaths)-65
    print("time_strip_ind", time_strip_ind)
    considered_filepaths = filepaths[time_strip_ind:time_strip_ind+64]  
    print("len(considered_filepaths)", len(considered_filepaths)) 
    temporal_strip = torch.tensor([]).to(device)
    
    #random_corner = random.choices(range(len(x_inds)), k=1)[0]
    random_corner = len(x_inds)-1
    #random_corner = 0
    print("random_corner", random_corner)
    x_corner = x_inds[random_corner]
    y_corner = y_inds[random_corner]

    if(x_corner>labels.shape[0]-70):
        x_corner = x_corner-70
    if(y_corner>labels.shape[1]-70):
        y_corner = y_corner-70

    space_label = labels[x_corner:x_corner+64, y_corner:y_corner+64]
    space_label = torch.from_numpy(space_label).to(device)
    
    spatial_label_batches = torch.cat((spatial_label_batches, space_label.unsqueeze(0)), axis=0)
    for filepath in considered_filepaths:
        numpy_array = np.load(filepath)
        torch_tensor = torch.from_numpy(numpy_array).to(device)
        numpy_array = 0.0

        temporal_strip = torch.cat((temporal_strip, torch_tensor[x_corner:x_corner+64, y_corner:y_corner+64].unsqueeze(0)), axis=0)

    temporal_batches = torch.cat((temporal_batches, temporal_strip.unsqueeze(0)) , axis=0)

    print("temporal_batches.shape", temporal_batches.shape)
    print("spatial_label_batches.shape", spatial_label_batches.shape)

    array_lai = temporal_batches.cpu().numpy()
    #tifffile.imsave('/home/luser/stelar_3d/storage/per_crop_data_labels/'+vista_crop_dict[chosen_crop_types]+'/train'+vista_crop_dict[chosen_crop_types]+'.tif', array_lai)


    array_lab = spatial_label_batches.cpu().numpy()
    #tifffile.imsave('/home/luser/stelar_3d/storage/per_crop_data_labels/'+vista_crop_dict[chosen_crop_types]+'/lab'+vista_crop_dict[chosen_crop_types]+'.tif', array_lab)


time_strip_ind 179
len(considered_filepaths) 64
random_corner 1106625
temporal_batches.shape torch.Size([1, 64, 64, 64])
spatial_label_batches.shape torch.Size([1, 64, 64])
time_strip_ind 179
len(considered_filepaths) 64
random_corner 1106625
temporal_batches.shape torch.Size([2, 64, 64, 64])
spatial_label_batches.shape torch.Size([2, 64, 64])
time_strip_ind 196
len(considered_filepaths) 64
random_corner 1106625
temporal_batches.shape torch.Size([3, 64, 64, 64])
spatial_label_batches.shape torch.Size([3, 64, 64])
time_strip_ind 52
len(considered_filepaths) 64
random_corner 1106625
temporal_batches.shape torch.Size([4, 64, 64, 64])
spatial_label_batches.shape torch.Size([4, 64, 64])
time_strip_ind 75
len(considered_filepaths) 64
random_corner 1106625
temporal_batches.shape torch.Size([5, 64, 64, 64])
spatial_label_batches.shape torch.Size([5, 64, 64])
time_strip_ind 12
len(considered_filepaths) 64
random_corner 1106625
temporal_batches.shape torch.Size([6, 64, 64, 64])
spatial_label_bat

KeyboardInterrupt: 

In [35]:
random.choices(range(4), k=1)[0]

0